In [89]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pprint import pprint

# import keras
from keras.models import load_model

loading nn model

In [90]:
# To create the training, testing and validation datasets from a single dataset.
from sklearn.model_selection import train_test_split

# To create the neural network 
from keras.models import Sequential

# To add hidden layers as per requirements
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# To call the 
from keras.callbacks import ModelCheckpoint

In [91]:
from keras.optimizers import Adam
from keras.layers import BatchNormalization

adam = Adam(lr=0.001, beta_1=0.5)

nn=Sequential()

nn.add(Conv2D(filters=16, kernel_size=3, padding='valid', activation='relu' , input_shape = (45,45,3)))
nn.add(BatchNormalization())
nn.add(MaxPooling2D(pool_size=2))
nn.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
nn.add(BatchNormalization())
nn.add(MaxPooling2D(pool_size=2))
nn.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
nn.add(BatchNormalization())
nn.add(MaxPooling2D(pool_size=2))

nn.add(Flatten())
nn.add(Dense(500, activation='relu'))
nn.add(Dropout(0.5))
nn.add(Dense(23, activation='softmax'))

nn.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

#nn.summary()

In [92]:
nn.load_weights(r'C:\Users\rajneesh\Downloads\new project\wt2.hdf5')

# Run

In [167]:
im=cv2.imread(r"C:\Users\rajneesh\Downloads\new project\test19.jpg")
import detect
obj=detect.contour_detection(im)

In [168]:
# save images
indx=1
x_test=[]

for row_im in obj.all_contours_img:
    
    #temp_arr = []
    
    for im in row_im:
    
        # save to local
        cv2.imwrite(r"C:/Users/rajneesh/Downloads/new project/detected/correct/"+str(indx)+".jpg",im)

#         # append to testing list
#         im = cv2.resize(im,(45,45))
#         im = [im,im,im]
#         im = np.stack(im,axis=2)
# #         x_test.append(im)
#         temp_arr.append(im)
        indx += 1
    #print(indx)
        
#     x_test.append(temp_arr)

# x_test=np.stack(x_test,axis=0)       

In [169]:
labels=['0','1','2','3','4','5','6','7','8','9','a','b','c','d','y','z','+','-','x','/','=','(',')']

In [170]:
all_exp = []
for row_im in obj.all_contours_img :
    string = ''
    for im in row_im :
        im = cv2.resize(im,(45,45))
        im = im/255
        a = [im,im,im]
        a = np.stack(a,axis=2)
        a = [a]
        a = np.stack(a,axis=0)
        #print(a.shape)
        #print(np.argmax(nn.predict(a)))
        string += labels[np.argmax(nn.predict(a))]
    all_exp.append(string)    

In [171]:
all_exp

['+z=', '']

In [35]:
x_test.shape()

AttributeError: 'list' object has no attribute 'shape'

In [19]:
# to remove ambiguity between 'x' and '*'
def x_mul(digits,i):
    
    n=len(digits)
    
    #valid output of nn corresponding to numbers
    val_num=range(0,11)

    if i==n-1:
        return 'x'
    else:
        if (digits[i+1] in val_num) or digits[i+1]=='(':
            return '*'
        else:
            return 'x'

In [20]:
# digitsing equation
eq=[]
for row in x_test:
    result=nn.predict(row)
    digits=result.argmax(axis=1)

    temp_eq=''
    #iterating backwards
    n=len(digits)
    for indx,x in enumerate(digits[::-1]):
        if x in list(range(0,10)):
            temp_eq+=str(x)
        elif x==10:
            temp_eq+=str('+')
        elif x==11:
            temp_eq+=str('-')
        elif x==12 or x=='x':
            char=str(x_mul(digits,n-indx-1))
            temp_eq+=char
        elif x==13:
            temp_eq+=str('/')
        elif x==14:
            temp_eq+=str('(')
        else:
            temp_eq+=str(')')
    temp_eq=temp_eq[::-1]
    eq.append(temp_eq)

In [21]:
eq

[]

# Solving the equation

In [36]:
class Stack:#creating stack class
    def __init__(self):
        self.items = []
        self.topp=-1
 
    def is_empty(self):
        return self.items == []
 
    def push(self, data):
        self.items.append(data)
        self.topp +=1
 
    def pop(self):
        self.topp -=1
        return self.items.pop()

    def top(self):
        return self.items[self.topp]
    
def pri(st):#returns priority of operator in stack
    if (st=="+"):
        return 2
    elif( st=="("):
        return 0
    elif( st=="-"):
        return 2
    elif(st=="*"):
        return 3
    elif(st=="/"):
        return 3

def chk_pr(d1,d2):#compares priority of 2 operators
    return (pri(d1)<=pri(d2))

def oper(d1,d2,d3):#to perfor operation
    if(d3=="+"):
        return (d1+d2)
    if(d3=="-"):
        return (d1-d2)
    if(d3=="*"):
        return (d1*d2)    
    if(d3=="/"):
        return (d1/d2)


def solve(exp):#to solve the expression. takes list of string with "(" and ")" at start and end respectively
    s = Stack()#strng = input()
    pexp=[]#postfix epression
    a=""
    for i in exp:#converting expression into postfix
        if (i=="("):
            s.push(i)
        elif (i.isdigit() or (len(i)>1 and i[1].isdigit())):
            pexp.append(i)
        elif (i==")"):
            a=s.pop()
            while(a!="("):
                pexp.append(a)
                a=s.pop()
        elif (i in "+-*/"):
            if (chk_pr(i,s.top())):
                while (chk_pr(i,s.top())):
                    pexp.append(s.pop())
            s.push(i)
    d1=0
    d2=0
    for i in pexp:#evaluating postfix expression
        if(i.isdigit() or (len(i)>1 and i[1].isdigit())):
            s.push(i)
        else:
            d1=s.pop()
            d2=s.pop()
            s.push(oper(int(d2),int(d1),i))
    return(s.pop())#returning solution of expression from the stack


def solver_main(exp):#call this for solving equation.It will automatically call other functions.It decides whether to solve equation or return true/false and calls solve()to solve the equations.
    
    exp.insert(0,"(")#inserting ( at the begining of expression
    exp.append(")")#inserting ) at the end of expression

    z=0#to check for = and default is no =
    f=0
    exp1=[]#for dividing into 2 expressions when there is an = sign in expression
    exp2=[]
    x=0#to save results of 2 expressions
    y=0

    for i in range(3,len(exp)-1):#check for right side of = sign. if nothing then z=0 otherwise z=1. i is starting from 3 for condition when = sign is at start
        if (((exp[i].isdigit() or (len(exp[i])>1 and exp[i][1].isdigit())) or exp[i]=="(") and exp[i-1]=="="):
            z=1
        

    if (z==1):
        for i in exp:#splitting expression into 2 expressions when there is an = sign
            if(i=="="):
                f=1
                exp1.append(")")#ending expression 1
                exp2.append("(")#starting expression 2
                continue
            if(f==0):
                exp1.append(i)
            else:
                exp2.append(i)
        x=solve(exp1)#solving both expressions separating exp individually
        y=solve(exp2)
        
        if(int(x)==int(y)):#checking for equality
            print ("TRUE")
        else:
            print("FALSE")

    else:
        print(solve(exp))

In [23]:
# Returns True if the input paramater is an operator, otherwise returns False.
def isOperator(x) :
    op = ['+' , '-' , '*' , '/' , '=','(',')']
    if x in op :
        return True
    else :
        return False            

# Returns a list that contains the expression. Each item in the list is either an
# operator or an operand .
def createExpression(eq) :
    
    exp_list = []
    buffer = ''
    
    for e in range(len(eq)) :
        
        if isOperator(eq[e]) :
            if e == 0 :
                exp_list.append(eq[e])
            else :
                exp_list.append(buffer)
                exp_list.append(eq[e])
            buffer = ''
        
        else :
            buffer += eq[e]
        #print(buffer)
    exp_list.append(buffer)
    
        
    if exp_list[0] == '-' :#to remove - sign at the start when number is next
        exp_list.pop(0)
        exp_list[0] = '-'+exp_list[0]
    
    while '' in exp_list :#remove empty strings
        exp_list.remove('')
        
    if(exp_list[0]=='-' and exp_list[1]=='('):#To remove - sign at the start when ( is next
        i=2
        while (exp_list[i]!=')'):
            if(exp_list[i].isdigit() or (len(exp_list[i])>1 and exp_list[i][1].isdigit())):
                exp_list[i]=str(int(exp_list[i])*(-1))
            i+=1
        exp_list.pop(0) 
        
    a=len(exp_list)-1#To remove - sign when ( is before it
    i=1
    while(i<a):
        if (exp_list[i]=='-' and exp_list[i-1]=='('):
            exp_list[i+1]=str(int(exp_list[i+1])*(-1))
            exp_list.pop(i)
            a-=1
        i+=1
    return exp_list        

In [37]:
for i in eq:
    solver_main(createExpression(i))

# solve linear eq

In [25]:
def isOperator(x) :
    if x in['+','-','*','/'] :
        return True
    else :
        return False
    
def coefficients(eq) :
    buffer = ''
    sign = 1
    # Using zz as the key for constant 
    coeff = {'zz':0}
    
    for e in eq :
        # We need variables to be on the left hand side of the '=', so the coefficients right to the '='
        # has to be negative
        if e == '=' :
            sign = -1
            # If a constant is just before the '=' we need to store it in the coefficient sum
            if buffer != '' :
                coeff['zz'] += int(buffer)
                buffer = ''
            #print('=')
            continue
        
        # If variable is encountered
        if e.isalpha() : 
            # If buffer is empty, it means its coefficient is 1.
            if buffer == '' :
                val = 1
            else :
                # If buffer is '-' or '+', it means the coefficient is 1. 
                if isOperator(buffer) :
                    buffer += '1'
                val = sign*int(buffer)
            # If the variable is encountered first time   
            if e not in coeff.keys() :
                coeff[e] = val
            # If the variable is encountered again    
            else :
                coeff[e] += val
            buffer = ''    
        # If a digit or an operator is encountered
        else :
            # Operator is encountered and buffer is empty
            if isOperator(e) and buffer == '' :
                buffer += e
            # A digit is encountered    
            elif not isOperator(e) :
                buffer += e
            # An operator is encountered and buffer is not empty    
            elif isOperator(e) and buffer != '' :
                coeff['zz'] += sign*int(buffer)
                buffer = e
        #print(e , coeff , buffer)    
    
    # We need the constant term on the right side of '='
    coeff['zz'] *= -1
    coeff['zz'] += int(buffer)
    
    #print(buffer)
    return coeff

In [26]:
# Takes a list of dictionary as an input and returns the solution in a numpy array as [x_sol , y_sol , ...]. 

# coeffs is list of dictionaries containing the coefficients
def solveLinearEq(coeffs) :
    
    var = []
    const = []
    
    for i in range(len(coeffs)) :
        all_coeffs = list(coeffs[i].keys())
        all_coeffs.sort()
        #print(all_coeffs , type(all_coeffs))
        temp = []
        for j in range(len(all_coeffs)-1) : 
            #print('#'*10)
            #print(j , all_coeffs[j])
            temp.append(coeffs[i][all_coeffs[j]])
        var.append(temp)
        const.append(coeffs[i][all_coeffs[-1]])
        
    ab = np.array(var)
    c = np.array(const)

    sol = np.linalg.solve(ab , c)

    return sol

In [27]:
import numpy as np

a = [coefficients('x-y+z=0'),coefficients('x+y+z=0'),coefficients('x-y-z=0')]
solveLinearEq(a)

array([ 0.,  0., -0.])